<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Demo 7.2.4: Ensemble Methods

INSTRUCTIONS:

- Run the cells
- Observe and understand the results
- Answer the questions

This is an excerpt from the [Introduction to Ensembling/Stacking in Python
](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python) by **Anisotropic** on **Kaggle**.
Credits also to:
- **Faron**: Mathias Müller, Data Scientist @ H2O.ai, Diplom (eq. MSc.) in Computer Science | HU-Berlin (zhryyrezng84@tznvy.pbz)
- **Sina**: AI Student at IUSTTehran, Tehran Province, Iran (https://sinakhorami.github.io)

## Dataset: Titanic
The full Titanic dataset is available from the Department of Biostatistics at the [Vanderbilt University School of Medicine](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.csv) in several formats. 

The [Encyclopedia Titanica](https://www.encyclopedia-titanica.org/) is the website of reference regarding the Titanic. It contains all the facts, history, and data surrounding the Titanic, including a full list of passengers and crew members. 

The Titanic dataset is also the subject of the introductory competition on [Kaggle.com](https://www.kaggle.com/c/titanic). Requires opening an account.

### Overview
The data has been split into two groups:
- Training set (`train.csv`)
- Test set (`test.csv`)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

### Data Dictionary
- **PassengerID**: A column added by Kaggle to identify each row and make submissions easier
- **Survived**: Whether the passenger survived or not and the value we are predicting (`0`=No, `1`=Yes)
- **Pclass**: The class of the ticket the passenger purchased (`1`=1st, `2`=2nd, `3`=3rd)
- **Sex**: The passenger's sex
- **Age**: The passenger's age in years
- **SibSp**: The number of siblings or spouses the passenger had aboard the Titanic
- **Parch**: The number of parents or children the passenger had aboard the Titanic
- **Ticket**: The passenger's ticket number
- **Fare**: The fare the passenger paid
- **Cabin**: The passenger's cabin number
- **Embarked**: The port where the passenger embarked `C`=Cherbourg, `Q`=Queenstown, `S`=Southampton)

### Variable Notes
- **pclass**: A proxy for socio-economic status (SES)
    - _1st_ = Upper
    - _2nd_ = Middle
    - _3rd_ = Lower
- **age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
- **sibsp**: The dataset defines family relations in this way
    - _Sibling_ = brother, sister, stepbrother, stepsister
    - _Spouse_ = husband, wife (mistresses and fiancés were ignored)
- **parch**: The dataset defines family relations in this way
    - _Parent_ = mother, father
    - _Child_ = daughter, son, stepdaughter, stepson
    - Some children travelled only with a nanny, therefore `parch=0` for them

In [ ]:
## Import Libraries

import re

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

## Load data

In [ ]:
## Loading the dataset

# Load in the train and test datasets
train = pd.read_csv('../../data/titanic_train.csv')
test = pd.read_csv('../../data/titanic_test.csv')

In [ ]:
## Check the data

# About train
print(train.info())
train.head()

In [ ]:
# About test
print(test.info())
test.head()

## Data Cleaning, Exploration and Feature Engineering

In [ ]:
# Extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ''

full_data = [train, test]
# calculate quantiles
fare_quant = train.Fare.quantile([.25, .5, .75])
age_quant = train.Age.quantile([0.2, 0.4, 0.6, 0.8])

verbose = True

# Gives the length of the name
for dataset in full_data:
    dataset['Name_length'] = dataset['Name'].apply(len)

# Feature that tells whether a passenger had a cabin on the Titanic
for dataset in full_data:
    dataset['Has_Cabin'] = dataset['Cabin'].apply(lambda x: 0 if type(x) == float else 1)

# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
# train['CategoricalFare'] = pd.qcut(train['Fare'], 4, duplicates = 'drop')

# Create a New feature CategoricalAge
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    dataset['Age'] = dataset['Age'].apply(lambda x: x if x is None else (np.random.randint(age_avg - age_std, age_avg + age_std)))
    dataset['Age'] = dataset['Age'].astype(int)
# train['CategoricalAge'] = pd.cut(train['Age'], 5, duplicates = 'drop')

# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

# Group all non-common titles into one single grouping 'Rare'
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {'Mr': 1, 'Miss': 2, 'Mrs': 3, 'Master': 4, 'Rare': 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
#     dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']                               = 0
#     dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
#     dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
#     dataset.loc[ dataset['Fare'] > 31, 'Fare']                                  = 3
#     dataset['Fare'] = dataset['Fare'].astype(int)
    for i in range(len(fare_quant) + 1):
        if i == 0:
            p = fare_quant.iloc[i]
            dataset.loc[ dataset['Fare'] <= p, 'Fare'] = i # 0
            if verbose:
                print('Categorising Fare')
                print('%d: > %7.4f and <= %7.4f ' % (i, dataset['Fare'].min(), p))
            continue
        if i == len(fare_quant):
            p_1 = fare_quant.iloc[i - 1]
            dataset.loc[ dataset['Fare'] > p_1, 'Fare'] = i # len(quant)
            if verbose:
                print('%d: > %7.4f' % (i, p_1))
            continue
        if i < len(fare_quant):
            p = fare_quant.iloc[i]
            p_1 = fare_quant.iloc[i - 1]
            dataset.loc[(dataset['Fare'] > p_1) & (dataset['Fare'] <= p), 'Fare'] = i
            if verbose:
                print('%d: > %7.4f and <= %7.4f' % (i, p_1, p))
    
    # Mapping Age
#     dataset.loc[ dataset['Age'] <= 16, 'Age']                          = 0
#     dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
#     dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
#     dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
#     dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4
    for i in range(len(age_quant) + 1):
        if i == 0:
            p = age_quant.iloc[i]
            dataset.loc[ dataset['Age'] <= p, 'Age'] = i # 0
            if verbose:
                print('Categorising Age')
                print('%d: > %7.4f and <= %7.4f ' % (i, dataset['Age'].min(), p))
            continue
        if i == len(age_quant):
            p_1 = age_quant.iloc[i - 1]
            dataset.loc[ dataset['Age'] > p_1, 'Age'] = i # len(quant)
            if verbose:
                print('%d: > %7.4f' % (i, p_1))
            continue
        if i < len(age_quant):
            p = age_quant.iloc[i]
            p_1 = age_quant.iloc[i - 1]
            dataset.loc[(dataset['Age'] > p_1) & (dataset['Age'] <= p), 'Age'] = i
            if verbose:
                print('%d: > %7.4f and <= %7.4f' % (i, p_1, p))

    verbose = False

In [ ]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
# train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [ ]:
print(train.info())
train.head()

In [ ]:
print(test.info())
test.head()

## Pearson Correlation Heatmap
Check correlation of the features to see how related one feature is to the next.

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize = (14, 12))
plt.title('Pearson Correlation of Features', size = 15)
sns.heatmap(train.astype(float).corr(),
            linewidths = 0.1,
            vmax = 1.0,
            square = True,
            cmap = colormap,
            linecolor = 'white',
            annot = True)
plt.show()

**QUESTION**: What does the chart say?

The Pearson Correlation plot can tell that there are not too many features strongly correlated with one another. 

This is good because:
- this means that there is not much redundant or superfluous data in the training set
- each feature carries with it some unique information

The two most correlated features are `FamilySize` and `Parch` (Parents and Children)

## Pairplots

In [ ]:
features = ['Survived', 'Pclass', 'Sex', 'Age', 'Parch', 'Fare', 'Embarked', 'FamilySize', 'Title']
g = sns.pairplot(
    train[features],
    hue = 'Survived',
    palette = 'seismic',
    size = 1.2,
    diag_kind = 'kde',
    diag_kws = dict(shade = True),
    plot_kws = dict(s = 10))
g.set(xticklabels = [])
plt.show()

## Ensembling & Stacking models

#### Helpers via Python Classes
Here we invoke the use of Python's classes to help make it more convenient. In short, a class helps to extend some code/program for creating objects as well as to implement functions and methods specific to that class.

In the code below, a class `SklearnHelper` allows to extend the inbuilt methods (such as train, predict and fit) common to all the `Scikit-Learn` classifiers. This cuts out redundancy as it avoids to write the same methods many times when invoking many different classifiers.

In [ ]:
# Some common parameters
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(n_splits = NFOLDS)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def predict_proba(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self, x, y):
        return self.clf.fit(x, y)
    
    def feature_importances(self, x, y):
        return self.clf.fit(x, y).feature_importances_

### Out-of-Fold Predictions
Stacking uses predictions of base classifiers as input for training to a second level model.

However one cannot simply train the base models on the full training data, generate predictions on the full test set and then output these for the second level training.

This runs the risk of the base model predictions already having "seen" the test set and therefore overfitting when feeding these predictions.

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain, ))
    oof_test = np.zeros((ntest, ))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis = 0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

### Generating the Base First Level Models
The first level classification models can all be conveniently invoked via the `Scikit-Learn` library and are listed as follows:

- Random Forest classifier
- Extra Trees classifier
- AdaBoost classifier
- Gradient Boosting classifier
- Support Vector classifier

#### Parameters

A quick summary of the parameters:

- **n_jobs**: Number of cores used for the training process. If set to -1, all cores are used.
- **n_estimators**: Number of classification trees in your learning model (set to 10 per default).
- **max_depth**: Maximum depth of tree, or how much a node should be expanded. Beware if set to too high a number would run the risk of overfitting as one would be growing the tree too deep.
- **verbose**: Controls whether to output any text during the learning process. A value of 0 suppresses all text while a value of 3 outputs the tree learning process at every iteration.

Please check out the full description via the official [Scikit-Learn](http://scikit-learn.org/) website.

In [ ]:
# Set the parameters for the classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
    'warm_start': True,
#     'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'verbose': 0
}

# Extra Trees parameters
et_params = {
    'n_jobs': -1,
    'n_estimators': 500,
#     'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
#     'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025,
    'probability' : True
}

In [ ]:
# Create the objects that represent our the models
rf  = SklearnHelper(clf = RandomForestClassifier,     seed = SEED, params = rf_params)
et  = SklearnHelper(clf = ExtraTreesClassifier,       seed = SEED, params = et_params)
ada = SklearnHelper(clf = AdaBoostClassifier,         seed = SEED, params = ada_params)
gb  = SklearnHelper(clf = GradientBoostingClassifier, seed = SEED, params = gb_params)
svc = SklearnHelper(clf = SVC,                        seed = SEED, params = svc_params)

In [ ]:
# Create Numpy arrays to feed into the models
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis = 1)
x_train = train.values
x_test = test.values

In [ ]:
# Create our OOF (Out-of-Fold) train and test predictions
# These base results will be used as new features
et_oof_train,  et_oof_test  = get_oof(et,  x_train, y_train, x_test) # Extra Trees
rf_oof_train,  rf_oof_test  = get_oof(rf,  x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train,  gb_oof_test  = get_oof(gb,  x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc, x_train, y_train, x_test) # Support Vector Classifier

print('Training is complete')

### Feature importance from the different classifiers
The importance of the various features in the training and test sets is available from the first level classifiers with one very simple line of code.

As per the `Scikit-Learn` documentation, most of the classifiers are built in with an attribute which returns feature importances by refering to the `.featureimportances_` attribute.

**NOTE**: SVC does not have the `.featureimportances_` attribute.

In [ ]:
# Create a dataframe with features
cols = train.columns.values
feature_dataframe = pd.DataFrame({
    'Feature':        cols,
    'Random Forest':  rf.feature_importances(x_train, y_train),
    'Extra Trees':    et.feature_importances(x_train, y_train),
    'AdaBoost':       ada.feature_importances(x_train, y_train),
    'Gradient Boost': gb.feature_importances(x_train, y_train)
})

In [ ]:
feature_dataframe

In [ ]:
fig = plt.figure(figsize = (15, 6))

ind = np.arange(len(cols)) 
width = 0.2

for i, m in enumerate(feature_dataframe.columns[1:]):
    plt.bar(ind + ((i - 1 - width) * width), feature_dataframe[m], width, label = m)
    
plt.title('Feature Importance')
plt.xticks(ind, cols, rotation = 45)
plt.ylabel('Score')
plt.legend()
plt.show()

In [ ]:
# Create a new column containing the average of values
feature_dataframe['Mean'] = feature_dataframe.mean(axis = 1) # axis = 1 computes the mean row-wise
feature_dataframe

In [ ]:
fig = plt.figure(figsize = (15, 6))

ind = np.arange(len(cols)) 
width = 0.15

for i, m in enumerate(feature_dataframe.columns[1:]):
    plt.bar(ind + ((i - 2) * width), feature_dataframe[m], width, label = m)

plt.title('Feature Importance')
plt.xticks(ind, cols, rotation = 45)
plt.ylabel('Score')
plt.legend()
plt.show()

In [ ]:
def show_summary_report(actual, prediction):

    if isinstance(actual, pd.Series):
        actual = actual.values
    if actual.dtype.name == 'object':
        actual = actual.astype(int)
    if prediction.dtype.name == 'object':
        prediction = prediction.astype(float)
        
    prediction_int = np.round(prediction)

    accuracy_ = accuracy_score(actual, prediction_int)
    precision_ = precision_score(actual, prediction_int)
    recall_ = recall_score(actual, prediction_int)
    roc_auc_ = roc_auc_score(actual, prediction)

    print('Accuracy : %.4f [TP / N] Proportion of predicted labels that match the true labels. Best: 1, Worst: 0' % accuracy_)
    print('Precision: %.4f [TP / (TP + FP)] Not to label a negative sample as positive.        Best: 1, Worst: 0' % precision_)
    print('Recall   : %.4f [TP / (TP + FN)] Find all the positive samples.                     Best: 1, Worst: 0' % recall_)
    print('ROC AUC  : %.4f                                                                     Best: 1, Worst: < 0.5' % roc_auc_)
    print('-' * 107)
    print('TP: True Positives, FP: False Positives, TN: True Negatives, FN: False Negatives, N: Number of samples')

    # Confusion Matrix
    mat = confusion_matrix(actual, prediction_int)

    # Precision/Recall
    precision, recall, _ = precision_recall_curve(actual, prediction_int)
    average_precision = average_precision_score(actual, prediction_int)
    
    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(actual, prediction)
    roc_auc = auc(fpr, tpr)

    # plot
    fig, ax = plt.subplots(1, 3, figsize = (18, 6))
    fig.subplots_adjust(left = 0.02, right = 0.98, wspace = 0.2)

    # Confusion Matrix
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False, cmap = 'Blues', ax = ax[0])

    ax[0].set_title('Confusion Matrix')
    ax[0].set_xlabel('True label')
    ax[0].set_ylabel('Predicted label')
    
    # Precision/Recall
    step_kwargs = {'step': 'post'}
    ax[1].step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
    ax[1].fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
    ax[1].set_ylim([0.0, 1.0])
    ax[1].set_xlim([0.0, 1.0])
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('2-class Precision-Recall curve')

    # ROC
    ax[2].plot(fpr, tpr, color = 'darkorange', lw = 2, label = 'ROC curve (AUC = %0.2f)' % roc_auc)
    ax[2].plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    ax[2].set_xlim([0.0, 1.0])
    ax[2].set_ylim([0.0, 1.0])
    ax[2].set_xlabel('False Positive Rate')
    ax[2].set_ylabel('True Positive Rate')
    ax[2].set_title('Receiver Operating Characteristic')
    ax[2].legend(loc = 'lower right')

    plt.show()
    
    return (accuracy_, precision_, recall_, roc_auc_)

### Summary for the First Level
**NOTE**: As the dataset is given from a Kaggle competition, the output values for the test set are **not** known. The summary report below are from the training data.

In [ ]:
# Keep the results in a dataframe
results = pd.DataFrame(columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'ROC_AUC'])

models = ['Random Forest', 'Extra Trees', 'AdaBoost', 'Gradient Boost', 'Support Vector Classifier']
for i, m in enumerate([rf, et, ada, gb, svc]):
    print('*' * (len(models[i]) + 4))
    print('* %s *' % models[i])
    print('*' * (len(models[i]) + 4))
    predictions = m.predict_proba(x_train)[:,1]
    # show the report
    accuracy_, precision_, recall_, roc_auc_ = show_summary_report(y_train, predictions)
    # keep the results
    results.loc[i] = {'Model': models[i], 
                      'Accuracy': accuracy_, 
                      'Precision': precision_,
                      'Recall': recall_,
                      'ROC_AUC': roc_auc_}
    print()

## Second Level Predictions from the First Level Output
### First level output as new features
One can think of building a new set of features from the first level predictions to be used as training data for the next classifier.

The code below have the first level predictions from the earlier classifiers as new columns and trains the next classifier from those.

In [ ]:
base_predictions_train = pd.DataFrame({
    'RandomForest': rf_oof_train.ravel(),
    'ExtraTrees': et_oof_train.ravel(),
    'AdaBoost': ada_oof_train.ravel(),
    'GradientBoost': gb_oof_train.ravel()
})
print('Rows: %d, Columns: %d' % base_predictions_train.shape)
base_predictions_train.head()

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize = (12, 12))
plt.title('Correlation Heatmap of the Second Level Training set', size = 15)
sns.heatmap(base_predictions_train.astype(float).corr(),
            linewidths = 0.1,
            vmax = 1.0,
            square = True,
            cmap = colormap,
            linecolor = 'white',
            annot = True)
plt.show()

In [ ]:
# Creates a new set of training and test data from the previous results results
xx_train = np.concatenate(
    (et_oof_train,
     rf_oof_train,
     ada_oof_train,
     gb_oof_train,
     svc_oof_train),
    axis = 1)
xx_test = np.concatenate(
    (et_oof_test,
     rf_oof_test,
     ada_oof_test,
     gb_oof_test,
     svc_oof_test),
    axis = 1)

## Second Level learning model via XGBoost

The **XGBoost** was built to optimize large-scale boosted tree algorithms.

The XGBoost parameters used in the model:

- **max_depth**: How deep to grow the tree. Runs the risk of overfitting if set to too high.
- **gamma**: Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger, the more conservative the algorithm will be.
- **eta**: Step size shrinkage used in each boosting step to prevent overfitting.

In [ ]:
gbm = xgb.XGBClassifier(
#     learning_rate = 0.02,
    n_estimators = 2000,
    max_depth = 4,
    min_child_weight = 2,
#     gamma = 1,
    gamma = 0.9,                        
    subsample = 0.8,
    colsample_bytree = 0.8,
    objective = 'binary:logistic',
    nthread = -1,
    scale_pos_weight = 1)

In [ ]:
gbm.fit(xx_train, y_train)
predictions = gbm.predict_proba(xx_train)[:,1]

In [ ]:
accuracy_, precision_, recall_, roc_auc_ = show_summary_report(y_train, predictions)
# keep the results
results.loc[len(results)] = {
    'Model': 'XGBoost', 
    'Accuracy': accuracy_, 
    'Precision': precision_,
    'Recall': recall_,
    'ROC_AUC': roc_auc_}
print()

In [ ]:
results



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



